In [2]:
from settings import CopilotSettings
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain_core.messages import HumanMessage, AIMessage
import pandas as pd
config = CopilotSettings()

In [3]:
df = pd.read_csv("data/SO_A_1-3FEB.xlsx - Sheet1.csv")
df.head()

,EncounterId,AdmissionNo,PatientName,Gender,BirthDate,Age,SubjectiveRemarksComplaint,SubjectiveRemarksAnamnesa,ObjectiveRemarks,ObjectiveTTV,AssessmentRemarks,ICD10,DoctorName,SpecialtyName,UnitName,OrganizationId,AdmissionDate
0,9E5279AD-A815-4999-8C20-3F86CAD26B68,OPA2402010001,KANATA ITO,MALE,2020-10-25 00:00:00.000,3Y 5M 1D,Mual-muntah dan diare.,Sejak Senin ada mual-muntah. Sudah periksa ke ...,Abd : BU + meningkat; supel; kembung,Temperature: °C\nBlood Pressure: / mmHg\nO2 Sa...,GEA,A09.94 GASTROENTERITIS NOS OR HAEMORRHAGIC OR ...,dr. Nico,GENERAL PRACTITIONER,SILOAM HOSPITALS LIPPO CIKARANG,4,2024-02-01 00:28:41.517
1,A51DFE9C-4471-4541-AA66-9C725A03F477,OPA2402010003,SARI ENDRIYANA,FEMALE,1988-01-29 00:00:00.000,36Y 1M 26D,Batuk dan pilek,Sejak sekitar sebulan terakhir batuk dahak sul...,Pulmo : Slem +/+,Temperature: °C\nBlood Pressure: / mmHg\nO2 Sa...,Bronchitis dd pneumonia,J18.99 PNEUMONIA UNSP; J40.91 BRONCHITIS NOS (...,dr. Nico,GENERAL PRACTITIONER,SILOAM HOSPITALS LIPPO CIKARANG,4,2024-02-01 06:39:22.370
2,4A706986-0CAC-4F11-97B9-F38B2254DCEC,OPA2402010025,DENG TAO,MALE,1971-07-13 00:00:00.000,52Y 8M 13D,"cough 2 days, slyme (+) fever yesterday histor...","cough 2 days, slyme (+) fever yesterday histor...",Keadaan Umum: Baik Kulit: Normal Kepala: Norma...,Temperature: °C\nBlood Pressure: / mmHg\nO2 Sa...,ISPA + Esofagitis + Fever,"J06.9 Acute upper respiratory infection, unspe...",dr. Fidelisa Cita Arini,GENERAL PRACTITIONER,SILOAM HOSPITALS LIPPO CIKARANG,4,2024-02-01 07:24:46.930
3,FA9866AE-FABE-4F03-97AD-DDAD743D8730,OPA2402010069,TANAKA YUKIMASA,MALE,1963-01-23 00:00:00.000,61Y 2M 3D,"os mengeluh sakit kepala, pusing keleyengenan ...","os mengeluh sakit kepala, pusing keleyengenan ...","Fairing: hiperemis ++, T1-T1 Thorax: sn ves, r...",Temperature: °C\nBlood Pressure: / mmHg\nO2 Sa...,HT primer + FA,I10.92 HYPERTENSION ARTERIAL (PRIMARY); J06.9 ...,dr. Sylvia Alicia Salim,GENERAL PRACTITIONER,SILOAM HOSPITALS LIPPO CIKARANG,4,2024-02-01 08:30:36.360
4,305296B6-9EFC-4367-9215-615EB49B3E85,OPA2402010080,EVI MERBINA BR TARIGAN,FEMALE,1992-08-03 00:00:00.000,31Y 7M 23D,pasien kontrol,"badan capek2, linu2",Keadaan Umum: Baik Kulit: Normal Kepala: Norma...,Temperature: °C\nBlood Pressure: / mmHg\nO2 Sa...,Myalgia,M79.199 MYALGIA SITE UNSP (PRIMARY),dr. Lyna Susanti,GENERAL PRACTITIONER,SILOAM HOSPITALS LIPPO CIKARANG,4,2024-02-01 08:37:15.407


In [4]:
llm = AzureChatOpenAI(
            deployment_name=config.OPENAI_DEPLOYMENT_NAME,
            api_key=config.OPENAI_API_KEY,
            openai_api_version=config.OPENAI_API_VERSION,
            azure_endpoint=config.OPENAI_API_ENDPOINT,
            temperature=0,
            max_tokens=8192,
            streaming=False,
            # callback_manager=[stream_handler],
        )

In [5]:
human_prompt = """
Patient visitation & doctor assessment:

Patient name:
{PatientName}

Gender:
{Gender}

Birth date:
{BirthDate}

Age:
{Age}

Subjective Remarks Complaint:
{SubjectiveRemarksComplaint}

Subjective Remarks Anamnesa:
{SubjectiveRemarksAnamnesa}

Objective Remarks:
{ObjectiveRemarks}

Assessment Remarks:
{AssessmentRemarks}
"""

human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

In [6]:
system_message = """
You are a physician. You will be provided with remarks from patient visitations and doctor assessments. You will create a single paragraph of free text of transcription summarizing all of this information. This text will represent a doctor's transcript based on those remarks. The remarks might be in Indonesian language and you will also generate the text in Indonesian.
"""

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system_message
        ),
        HumanMessagePromptTemplate.from_template(human_prompt),
    ]
)

prompt

ChatPromptTemplate(input_variables=['Age', 'AssessmentRemarks', 'BirthDate', 'Gender', 'ObjectiveRemarks', 'PatientName', 'SubjectiveRemarksAnamnesa', 'SubjectiveRemarksComplaint'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\nYou are a physician. You will be provided with remarks from patient visitations and doctor assessments. You will create a single paragraph of free text of transcription summarizing all of this information. This text will represent a doctor's transcript based on those remarks. The remarks might be in Indonesian language and you will also generate the text in Indonesian.\n")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['Age', 'AssessmentRemarks', 'BirthDate', 'Gender', 'ObjectiveRemarks', 'PatientName', 'SubjectiveRemarksAnamnesa', 'SubjectiveRemarksComplaint'], template='\nPatient visitation & doctor assessment:\n\nPatient name:\n{PatientName}\n\nGender:\n{Gender}\n\nBirth date:\n{BirthDate}\n\n

In [7]:
chain = (
        prompt
        | llm
    )

In [8]:
idx = 39
response = chain.invoke(
    {
        "PatientName": df["PatientName"].iloc[idx],
        "Gender": df["Gender"].iloc[idx],
        "BirthDate": df["BirthDate"].iloc[idx],
        "Age": df["Age"].iloc[idx],
        "SubjectiveRemarksComplaint": df["SubjectiveRemarksComplaint"].iloc[idx],
        "SubjectiveRemarksAnamnesa": df["SubjectiveRemarksAnamnesa"].iloc[idx],
        "ObjectiveRemarks": df["ObjectiveRemarks"].iloc[idx],
        "AssessmentRemarks": df["AssessmentRemarks"].iloc[idx],
    }
)
print(response.content)

Transkripsi dokter:

Pasien dengan nama Ryan Hidayat, laki-laki, lahir pada tanggal 12 Maret 1988, berusia 36 tahun, 0 bulan, dan 14 hari, datang dengan keluhan mual, muntah, dan nyeri ulu hati yang telah dirasakan sejak kemarin. Dari pemeriksaan objektif, keadaan umum pasien baik, kulit, kepala, mata, THT, leher, thorax, cor, pulmo, abdomen, hepar, lien, ekstremitas, dan genetalia dalam keadaan normal. Dari anamnesa dan pemeriksaan yang telah dilakukan, pasien didiagnosis dengan dyspepsia.


Transkrip dokter:

Pasien dengan nama Seshama, perempuan, berusia 21 tahun 2 bulan 28 hari, datang dengan keluhan demam. Pasien menginformasikan bahwa dua hari yang lalu ia pulang dari rumah sakit Hosana setelah dirawat karena demam dengue. Pada pagi hari ini, pasien kembali merasakan demam dan demam tersebut berlanjut hingga malam hari. Pasien mulai merasakan demam pada tanggal 23 Januari dan dirawat selama 3 hari 4 malam mulai tanggal 28 Januari. Pasien juga mengalami mual dan saat ini masih membawa obat-obatan seperti cefixime, domperidone, asetilsistein, dan lansoprazole. Dari pemeriksaan fisik, ditemukan tanda-tanda CM NTE + dan NT suprapubic +. Berdasarkan penilaian awal, pasien diduga mengalami febris yang mungkin disebabkan oleh infeksi saluran kemih (ISK).


In [49]:
!pip install streamlit-mic-recorder

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 3.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 6.0 MB/s eta 0:00:0000:0100:01
